In [1]:
# !pip install sentence-transformers evaluate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 787.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# !pip install git+https://github.com/google-research/bleurt.git

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-cv4yyfl3
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-cv4yyfl3
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456766 sha256=f6d91ca4df67e1a7629f1cc672512b95ec8b3d755645279720e87bb6a7bbb132
  Stored in directory: /tmp/pip-ephem-wheel-cache-9ko7mp5u/wheels/30/af/34/e148007788b060e4c76e7ecf68e70c692dff0f2632e62ac454
Successfully built BLEURT


In [3]:
import logging
import json
import torch
import numpy as np
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from evaluate import load

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Setup logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
if not logger.handlers:
    fh = logging.FileHandler('evaluation.log')
    fh.setLevel(logging.INFO)
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    ch.setFormatter(formatter)
    logger.addHandler(fh)
    logger.addHandler(ch)

In [6]:
# Detect device
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)
logger.info("--------  Start with Baseline Predictions  -------------")
logger.info(f'Device selected: {device}')

2025-04-29 20:58:40,618 - INFO - --------  Start with Baseline Predictions  -------------
INFO:__main__:--------  Start with Baseline Predictions  -------------
2025-04-29 20:58:40,620 - INFO - Device selected: cpu
INFO:__main__:Device selected: cpu


In [11]:
# === Configuration (replace argparse) ===
class Args:
    input_path = '/content/drive/MyDrive/HSG/NLP/Project NLP/Data/sample.json'
    submission_path = '/content/drive/MyDrive/HSG/NLP/Project NLP/Data/results_zeroshot_llama_3.1-8B-instruct-finetuned_formated.json'
    sim_threshold = 0.6 # recommened 0.6 -0.75 [0,1]
    bleurt_threshold = 0.2 # recommened 0.2 - 0.4 [-1,1]

args = Args()

In [8]:
# === Load model ===
sim_model = SentenceTransformer("stsb-mpnet-base-v2")
bleurt_model = load("bleurt", module_type="metric")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.49k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
# === Load data ===
with open(args.input_path) as f:
    reference = json.load(f)

with open(args.submission_path) as f:
    new = json.load(f)

In [13]:
# === Evaluate ===
predicted_labels = []
punctuations = []


for instance in new.keys():
    sim_punctuation = 0
    bleurt_punctuation = 0
    reference_set = [ref['cq'] for ref in reference[instance]['cqs']]

    if new[instance]['cqs'] != 'Missing CQs':
        cqs_check = [cq['cq'] for cq in new[instance]['cqs']]
        if len(cqs_check) != len(set(cqs_check)):
            logger.warning('There are repeated CQs in ' + instance)

        for i, line in enumerate(new[instance]['cqs']):

            sim_winner = None
            sim_sentence_embedding = sim_model.encode(line['cq'])
            sim_reference_embedding = sim_model.encode(reference_set)
            sims = sim_model.similarity(sim_sentence_embedding, sim_reference_embedding).tolist()[0]

            sim_winner = np.argmax(sims)
            sims_max_score = sims[sim_winner]

            if sims_max_score > args.sim_threshold:
                simlabel = reference[instance]['cqs'][sim_winner]['label']
                if simlabel == 'Useful':
                    sim_punctuation += 1/3
            else:
                label = 'not_able_to_evaluate'
            new[instance]['cqs'][i]['sim_label'] = simlabel
            new[instance]['cqs'][i]['sim_score'] = sims_max_score



            bleuert_winner = None
            bleurt_results = bleurt_model.compute(predictions=[line['cq']] * len(reference_set), references=reference_set)
            bleurt = bleurt_results['scores']

            bleuert_winner = np.argmax(bleurt)
            bleurt_max_score = bleurt[bleuert_winner]

            if bleurt_max_score > args.bleurt_threshold:
                bleurt_label = reference[instance]['cqs'][bleuert_winner]['label']
                if bleurt_label == 'Useful':
                    bleurt_punctuation += 1/3
            else:
                bleurt_label = 'not_able_to_evaluate'
            new[instance]['cqs'][i]['bleurt_label'] = bleurt_label
            new[instance]['cqs'][i]['bleurt_score'] = bleurt_max_score

            predicted_labels.append((simlabel, bleurt_label))

    else:
        predicted_labels.extend([('not_able_to_evaluate', 'not_able_to_evaluate'), ('not_able_to_evaluate', 'not_able_to_evaluate'), ('not_able_to_evaluate', 'not_able_to_evaluate')])

    new[instance]['sim_score'] = sim_punctuation
    new[instance]['bleurt_score'] = bleurt_punctuation
    punctuations.append((sim_punctuation, bleurt_punctuation))
    logger.info(f'{instance} score (based on similarity): {sim_punctuation:.2f}/1.00 score (based on bleurt): {bleurt_punctuation:.2f}/1.00')

2025-04-29 21:04:23,965 - INFO - CLINTON_1_1 score (based on similarity): 0.33/1.00 score (based on bleurt): 0.00/1.00
INFO:__main__:CLINTON_1_1 score (based on similarity): 0.33/1.00 score (based on bleurt): 0.00/1.00
2025-04-29 21:05:02,405 - INFO - Javier_84 score (based on similarity): 1.00/1.00 score (based on bleurt): 0.00/1.00
INFO:__main__:Javier_84 score (based on similarity): 1.00/1.00 score (based on bleurt): 0.00/1.00
2025-04-29 21:06:20,282 - INFO - TRUMP_125_1 score (based on similarity): 0.00/1.00 score (based on bleurt): 0.00/1.00
INFO:__main__:TRUMP_125_1 score (based on similarity): 0.00/1.00 score (based on bleurt): 0.00/1.00
2025-04-29 21:06:52,642 - INFO - travellots_133_1 score (based on similarity): 0.33/1.00 score (based on bleurt): 0.00/1.00
INFO:__main__:travellots_133_1 score (based on similarity): 0.33/1.00 score (based on bleurt): 0.00/1.00
2025-04-29 21:07:25,097 - INFO - Zewstain__641 score (based on similarity): 0.00/1.00 score (based on bleurt): 0.00/1.

In [14]:
# === Summary ===
logger.info('------ Summary Metrics ------')
logger.info(f'Distribution of labels: {Counter(predicted_labels)}')
logger.info(f'Distribution of punctuation: {Counter(punctuations)}')
total_sim, total_bleurt = 0.0, 0.0
for sim, bleurt in punctuations:
    total_sim += sim
    total_bleurt += bleurt

avg_sim = total_sim / len(punctuations) if punctuations else 0.0
avg_bleurt = total_bleurt / len(punctuations) if punctuations else 0.0

logger.info(f'Overall sim punctuation: {avg_sim:.4f}')
logger.info(f'Overall bleurt punctuation: {avg_bleurt:.4f}')

# === Save updated results with labels ===
output_path = args.submission_path[:-5] + f'_eval.json'
with open(output_path, 'w') as o:
    json.dump(new, o, indent=4)
logger.info(f"Saved labeled output to {output_path}")

2025-04-29 21:09:36,155 - INFO - ------ Summary Metrics ------
INFO:__main__:------ Summary Metrics ------
2025-04-29 21:09:36,158 - INFO - Distribution of labels: Counter({('Invalid', 'not_able_to_evaluate'): 6, ('Useful', 'not_able_to_evaluate'): 6, ('Unhelpful', 'not_able_to_evaluate'): 3})
INFO:__main__:Distribution of labels: Counter({('Invalid', 'not_able_to_evaluate'): 6, ('Useful', 'not_able_to_evaluate'): 6, ('Unhelpful', 'not_able_to_evaluate'): 3})
2025-04-29 21:09:36,160 - INFO - Distribution of punctuation: Counter({(0.3333333333333333, 0): 2, (0, 0): 2, (1.0, 0): 1})
INFO:__main__:Distribution of punctuation: Counter({(0.3333333333333333, 0): 2, (0, 0): 2, (1.0, 0): 1})
2025-04-29 21:09:36,164 - INFO - Overall sim punctuation: 0.3333
INFO:__main__:Overall sim punctuation: 0.3333
2025-04-29 21:09:36,166 - INFO - Overall bleurt punctuation: 0.0000
INFO:__main__:Overall bleurt punctuation: 0.0000
2025-04-29 21:09:36,175 - INFO - Saved labeled output to /content/drive/MyDrive